In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from skimage.transform import resize

Before running this notebook, you need to install the dhSegment package.

`pip install git+https://github.com/dhlab-epfl/dhSegment`

In [ ]:
import dh_segment
import tensorflow as tf

In [ ]:
from dh_segment.inference import LoadedModel

### Downloading pre-trained models

Download and unzip `model.zip` from https://github.com/dhlab-epfl/dhSegment/releases/tag/v0.2 for a page extraction mdoel.

Download and unzip `line_model.zip` from https://github.com/dhlab-epfl/FDH_Tutorial_ComputerVision_DeepLearning/releases for a texline extraction model.

In [ ]:
sess1 = tf.InteractiveSession()
with sess1.graph.as_default():
    model_page = LoadedModel("model/")
sess2 = tf.InteractiveSession(graph=tf.Graph())
with sess2.graph.as_default():
    model_textline = LoadedModel("polylines/")

In [ ]:
from skimage import io
img = io.imread("0167.jpg")
plt.figure(figsize=(10,10))
plt.imshow(img)

## Page extraction

In [ ]:
output_page = model_page.predict("0167.jpg")

In [ ]:
page_probs = output_page['probs'][0,:,:,1]
plt.figure(figsize=(10,10))
plt.imshow(page_probs)

In [ ]:
from dh_segment.post_processing import thresholding, cleaning_binary

page_mask = thresholding(page_probs, threshold=0.7)
page_mask = cleaning_binary(page_mask, kernel_size=7).astype(np.uint8)*255
plt.figure(figsize=(10,10))
plt.imshow(page_mask)

In [ ]:
from dh_segment.post_processing import boxes_detection


page_coords = boxes_detection.find_boxes(resize(page_mask, img.shape[:2]).astype(np.uint8), n_max_boxes=1)

In [ ]:
from dh_segment.io import PAGE
 
PAGE_info = PAGE.Page(image_width=img.shape[1], image_height=img.shape[0],
                      page_border=PAGE.Border(PAGE.Point.list_to_point(list(page_coords))))

In [ ]:
plot_img = img.copy()
PAGE_info.draw_page_border(plot_img, autoscale=True, fill=False, thickness=15)
plt.figure(figsize=(10,10))
plt.imshow(plot_img)

## Texline detection

In [ ]:
output_textline = model_textline.predict("0167.jpg")

In [ ]:
textline_probs = output_textline['probs'][0,:,:,1]
plt.figure(figsize=(10,10))
plt.imshow(textline_probs)

In [ ]:
from dh_segment.post_processing import cleaning_probs, thresholding, hysteresis_thresholding

textline_probs2 = cleaning_probs(textline_probs, 2)
extracted_page_mask = np.zeros(textline_probs.shape, dtype=np.uint8)
PAGE_info.draw_page_border(extracted_page_mask, color=(255,))
textline_mask = hysteresis_thresholding(textline_probs2, low_threshold=0.3, high_threshold=0.6,
                                        candidates_mask=extracted_page_mask>0
                                       )

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(textline_mask)

In [ ]:
from dh_segment.post_processing import line_vectorization

lines = line_vectorization.find_lines(resize(textline_mask, img.shape[:2]))

In [ ]:
text_region = PAGE.TextRegion()
text_region.text_lines = [PAGE.TextLine.from_array(line) for line in lines]
PAGE_info.text_regions.append(text_region)

In [ ]:
plot_img = img.copy()
PAGE_info.draw_page_border(plot_img, autoscale=True, fill=False, thickness=15)
PAGE_info.draw_lines(plot_img, autoscale=True, fill=False, thickness=15, color=(0,255,0))
plt.figure(figsize=(15,15))
plt.imshow(plot_img)